In [1]:
import sys
sys.path.append('../scripts/')
from mcl import*

Duplicate key in file WindowsPath('C:/Users/tauyu/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 257 ('font.family:  IPAexGothic')


<IPython.core.display.Javascript object>

In [2]:
class ResetMcl(Mcl): ###resetmcl3 （12行目以下）
    def __init__(self, envmap, init_pose, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}, \
                 distance_dev_rate=0.14, direction_dev=0.05, alpha_threshold=0.001): #alpha_thresholdを追加
        super().__init__(envmap, init_pose, num, motion_noise_stds, distance_dev_rate, direction_dev)
        self.alpha_threshold = alpha_threshold #追加
        
        
    def random_reset(self):
         for p in self.particles:
                p.pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
                p.weight = 1/len(self.particles)
                
    def sensor_resetting_draw(self, particle, landmark_pos, ell_obs, phi_obs): #追加
            ##パーティクルの位置を決める##
            psi = np.random.uniform(-np.pi, np.pi) #ランドマークからの方角を選ぶ
            ell = norm(loc=ell_obs, scale=(ell_obs*self.distance_dev_rate)**2).rvs() #ランドマークからの距離を選ぶ
            particle.pose[0] = landmark_pos[0] + ell*math.cos(psi)
            particle.pose[1] = landmark_pos[1] + ell*math.sin(psi)
            
            ##パーティクルの向きを決める##
            phi = norm(loc=phi_obs, scale=(self.direction_dev)**2).rvs() #ランドマークが見える向きを決める
            particle.pose[2] = math.atan2(landmark_pos[1]- particle.pose[1], landmark_pos[0]- particle.pose[0]) - phi
            
            particle.weight = 1.0/len(self.particles) #重みを1/Nに
            
    def sensor_resetting(self, observation): 
        d=list([obs[0][0] for obs in observation])
        if len(d)==2:
            nearest_obs=np.argmin([obs[0][0] for obs in observation]) #最小値の位置=最小値の位置を求める関数(idを含むセンサ値のリスト)
            values1, landmark_id1=observation[nearest_obs]#単なるセンサ値のリスト
            b = sorted(d).index(sorted(d)[1]) #ただのセンサ値
            values2, landmark_id2 = observation[b]   #bの型がおかしい可能性がある b(センサ値)の場所をobservation[]のかっこに入れる
            for p in self.particles:
                self.sensor_resetting_draw(p, self.map.landmarks[landmark_id1].pos, *values1)
                self.sensor_resetting_draw(p, self.map.landmarks[landmark_id2].pos, *values2)
                
            print(d)
            
            
        elif len(d)==1:
            nearest_obs3=np.argmin([obs[0][0] for obs in observation])
            values3, landmark_id3=observation[nearest_obs3]
            
            for p in self.particles:
                self.sensor_resetting_draw(p, self.map.landmarks[landmark_id3].pos, *values3)
                
            print(d)
                
    def observation_update(self, observation):        
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
            
        self.set_ml()

        if sum([p.weight for p in self.particles]) < self.alpha_threshold:
            self.sensor_resetting(observation) 
        else:
            self.resampling()

C:\Users\tauyu\anaconda\lib\site-packages\matplotlib\animation.py:889: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


In [3]:
def trial_trap_kidnap_phantom(animation):
    time_interval=0.1
    world=World(30,time_interval,debug=not animation)
    
    ##地図を生成して3つランドマークを追加##
    m=Map()
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)
    
    ##ロボットを作る##
    init_pose=np.array([np.random.uniform(-5.0,5.0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
    robot_pose=np.array([np.random.uniform(-5.0,5.0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
    pf=ResetMcl(m,init_pose,100)
    a=EstimationAgent(time_interval,0.2,10.0/180*math.pi,pf)
    r=Robot(robot_pose,sensor=Camera(m,phantom_prob=0.1),agent=a,color="red",expected_stuck_time=5,expected_escape_time=5)
    world.append(r)
    
    world.draw()
    return(r.pose,pf.ml.pose)

In [4]:
trial_trap_kidnap_phantom(True)

<IPython.core.display.Javascript object>

(array([ 2.32082854, -0.46934697,  1.43679948]),
 array([-1.51673149,  1.70623503,  2.53991169]))

In [5]:
if __name__ =='__main__':
    ok=0
    for i in range(1000):
        actual, estm=trial_trap_kidnap_phantom(False)
        diff=math.sqrt((actual[0]-estm[0])**2+(actual[1]-estm[1])**2)
        print(i,"真値:",actual,"推定値:",estm,"誤差:",diff)
        if diff<=1.0:
            ok+=1
            print(ok)

[1.89845178030161]
[1.2361657618599589, 3.6708844216665373]
[1.1896346743251]
[1.7039983936534624]
0 真値: [-4.56738902  0.75357735  3.00882168] 推定値: [-1.03421269  1.13606453  7.75835887] 誤差: 3.5538192797919255
[2.1120123907690744]
[2.3746574922377546]
[3.5590560408390726]
1 真値: [-0.82526489 -6.02611957  4.79344391] 推定値: [8.3366679  2.47463631 8.25573703] 誤差: 12.498154382193533
[5.32101850803833, 3.814550905882015]
[4.9182692864704025, 4.149795940514244]
[3.308746452467688]
[3.460243652122628]
[3.6535389471348085, 4.377721307909004]
[3.465663950304744, 3.265042284900447]
[4.823086880723975]
[3.7702867781119043]
[3.177066563461613]
[1.0263288817610512, 2.950552437267435]
[2.281577018290735]
[2.9389743422920667]
[5.432123278126539, 2.9549129361101856]
[4.116080892135544]
[3.279314220369112]
[2.064207721984535, 3.192729253916943]
[2.529486102565287]
[4.026865308293742]
[3.2084287198426926]
[1.222626406667659]
[3.7328014462809835]
2 真値: [-0.30691514  4.39452453  2.71090111] 推定値: [-4.71271934

C:\Users\tauyu\section_advanced_localization\../scripts\ideal_robot.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(4,4))


[1.2806721071784326]
[5.1559600316133345]
[1.171540946680124]
[1.0748089772129934, 4.761120901509163]
[1.1207364871712773]
[5.091349574846659]
[0.8949470125423225]
[1.2126018995180972, 3.949184571232124]
[1.1823070185356928]
[2.791354392763118]
[0.7960441871601323]
[1.1452386132984782, 5.73529660126202]
[1.2623173147459232]
[1.2807164806878297, 4.696213956756743]
[1.12663392472237, 2.291718432509026]
[1.2485091439395042]
[4.094844430654937]
[1.1436563789582266]
[3.7273040651206673]
[0.685252904271056]
[6.546745344045955]
[3.190614738188801]
[1.881662576518234]
20 真値: [-4.54259137  1.9820722  -1.09658193] 推定値: [-4.77497282  3.51218799 -1.00741267] 誤差: 1.5476612887151326
[4.320968286537008]
[4.002777420975877, 4.133216138214897]
[3.7189505053566485]
[4.794095227153446]
[4.090222696716614]
[2.1303601781941834]
[4.268171059526316]
[3.670857627760626, 3.67747131547721]
[3.7780653103406725]
[4.283143934941837, 1.5441317286701561]
[3.626849384615462, 4.845690933744362]
[4.031616167847315]
[3.